<a href="https://colab.research.google.com/github/RobelDawit/CS540/blob/main/RAG_Vector_store_chroma.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#install dependancies

In [1]:
!pip install -qU pinecone-client langchain openai pypdf unstructured langchain-community tiktoken langchain-openai chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 20.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 40.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.7/300.7 kB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 56.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 40.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 42.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.7/54.7 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 29.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

#setup import

In [2]:
import pinecone
from pinecone import Pinecone, ServerlessSpec
import os, getpass
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain_openai import OpenAI
import textwrap


#Mount drive to store vectorstore

In [3]:
from google.colab import drive
drive.mount('/content/drive')

# Save to Drive

book_dir = '/content/drive/MyDrive/Books'
persist_directory = '/content/drive/MyDrive/chroma_db'
book_name = book_dir +'/Aircraft Structures By Megson ( PDFDrive ).pdf'

Mounted at /content/drive


#Get API keys

In [4]:
def _set_env(var: str, force_update=False):

    if force_update or not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("OPENAI_API_KEY", force_update=True)

OPENAI_API_KEY: ··········


#upload book or document in PDF format

In [6]:
pdf_loader = PyPDFLoader("/content/drive/MyDrive/books/Aircraft Structures By Megson ( PDFDrive ).pdf")
documents = pdf_loader.load()

In [7]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
split_docs = text_splitter.split_documents(documents)

#Create embeddings

In [8]:
embeddings = OpenAIEmbeddings(openai_api_key=os.getenv("OPENAI_API_KEY"))

#Save vector store using chroma

In [19]:
vector_store = Chroma.from_documents(split_docs, embeddings, persist_directory=persist_directory)

#Query your book

In [40]:
retriever = vector_store.as_retriever(search_kwargs={"k": 10})
qa_chain = RetrievalQA.from_chain_type(llm=OpenAI(), chain_type="stuff", retriever=retriever)

query = "give me a high level overview of the most important structures (spars, stringers, frames, bulkheads etc...) on a loaded aircraft in flight and a breif overview of what they do?"
response = qa_chain.run(query)
wrapped_response = textwrap.fill(response, width=100)

print("Answer:\n", wrapped_response)

Answer:
  The most important structures on a loaded aircraft in flight are the spars, stringers, frames, and
bulkheads. These structures work together to provide strength and support to the aircraft's wings
and fuselage. The spars are long, horizontal beams that run through the length of the wings and
carry the majority of the wing's weight. The stringers are thin, longitudinal beams that run along
the length of the fuselage and provide additional support and stiffness to the aircraft's skin. The
frames are vertical structures that extend across the fuselage and help to distribute the weight and
stresses of the aircraft. Bulkheads are strong, reinforced walls that separate different sections of
the aircraft, such as the passenger area and the engine compartment. These structures are designed
to withstand the forces and pressures experienced during flight and work together to keep the
aircraft stable and safe.
